# LangChain - LLM 应用开发框架教程

欢迎来到 LangChain 教程！LangChain 是一个强大的开源框架，旨在简化基于大型语言模型 (LLM) 的应用程序的开发。它提供了一系列模块化组件、链 (Chains) 和代理 (Agents)，使得开发者能够轻松地将 LLM 与外部数据源、计算资源和 API 连接起来，构建更复杂、更有用的 AI 应用。

**为什么使用 LangChain？**

1.  **模块化与可组合性**: 提供标准化的接口和可组合的构建块 (Models, Prompts, Indexes, Chains, Memory, Agents)，方便灵活构建应用。
2.  **与 LLM 和外部工具集成**: 轻松连接不同的 LLM 提供商 (OpenAI, Hugging Face Hub, Anthropic 等) 以及各种工具 (搜索、计算器、数据库、API)。
3.  **数据感知**: 帮助 LLM 连接到私有或特定领域的数据源，实现基于特定知识的问答或内容生成 (RAG)。
4.  **状态管理**: 提供 Memory 组件来维护对话历史或应用状态。
5.  **Agentic 应用**: 支持构建能够自主规划和执行任务的智能代理。
6.  **活跃的社区和生态**: 快速发展，拥有大量示例和第三方集成。

**核心组件概览 (我们将重点介绍其中一些):**
*   **Models**: 与语言模型交互 (LLMs, ChatModels, Text Embedding Models)。
*   **Prompts**: 管理和优化模型输入 (Prompt Templates, Example Selectors)。
*   **Indexes**: 构建和查询外部数据 (Document Loaders, Text Splitters, Vector Stores, Retrievers)。
*   **Chains**: 将多个组件按顺序或逻辑组合起来 (LCEL - LangChain Expression Language, `SequentialChain` 等)。
*   **Memory**: 在 Chain 或 Agent 调用之间保持状态 (对话历史)。
*   **Agents & Tools**: 让 LLM 使用工具来执行动作。

**本教程将涵盖 LangChain 的核心概念和基础用法：**

1.  安装与设置 (API Keys)
2.  Models: 与 LLM 和 Chat Model 交互
3.  Prompts: 使用 Prompt Templates
4.  LCEL (LangChain Expression Language): 构建简单的 Chain
5.  Indexes: 文档加载、分割、嵌入和向量存储 (基础)
6.  Retrievers: 从索引中检索信息
7.  构建简单的 RAG (Retrieval-Augmented Generation) Chain
8.  (简介) Memory
9.  (简介) Agents & Tools

## 1. 安装与设置

首先，安装 LangChain 核心库。你还需要安装你想要使用的 LLM 提供商的库（例如 `openai`）和可能的其他集成库（如 `tiktoken` 用于 OpenAI 分词，`faiss-cpu` 或 `chromadb` 用于向量存储）。

```bash
pip install langchain openai tiktoken python-dotenv

# 如果需要使用向量数据库 (FAISS 示例)
pip install faiss-cpu # 或者 faiss-gpu 如果有 GPU 和 CUDA

# 如果需要使用 Hugging Face 模型
# pip install huggingface_hub transformers sentence-transformers
```

**设置 API Keys**: 
许多 LLM 服务（如 OpenAI）需要 API Key。最佳实践是**不要将 Key 硬编码在代码中**。推荐使用环境变量或 `.env` 文件。
1.  创建一个名为 `.env` 的文件在你的项目根目录。
2.  在 `.env` 文件中添加你的 Key: `OPENAI_API_KEY='your_api_key_here'`
3.  使用 `python-dotenv` 库来加载它。

In [ ]:
import langchain
import os
from dotenv import load_dotenv

# 尝试加载 .env 文件中的环境变量
load_success = load_dotenv() # 返回 True 如果 .env 文件被找到并加载
print(f"LangChain version: {langchain.__version__}")
print(f".env file loaded: {load_success}")

# 检查 OpenAI API Key 是否已设置 (从环境变量读取)
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print("OpenAI API Key found in environment variables.")
    # 为了安全，不直接打印 Key
    # print(f"OpenAI API Key starts with: {openai_api_key[:5]}...") 
    openai_available = True
else:
    print("OpenAI API Key not found. Please set it in your environment or a .env file to run OpenAI examples.")
    openai_available = False

# 可以在这里设置 Key，但不推荐用于共享代码
# import openai
# openai.api_key = "sk-..."
# os.environ["OPENAI_API_KEY"] = "sk-..." 

## 2. Models: 与 LLM 和 Chat Model 交互

LangChain 提供了与不同类型语言模型交互的标准接口。

*   **LLMs**: 基于文本补全的模型，输入是字符串，输出是字符串 (例如 OpenAI 的 `text-davinci-003` - 旧模型)。
*   **ChatModels**: 基于聊天消息的模型，输入是一系列消息 (System, Human, AI)，输出是一条 AI 消息 (例如 OpenAI 的 `gpt-3.5-turbo`, `gpt-4`)。
*   **Text Embedding Models**: 将文本转换为向量表示 (嵌入)。

In [ ]:
from langchain.chat_models import ChatOpenAI # 使用 Chat Model
# from langchain.llms import OpenAI # 使用旧的 LLM 接口
from langchain.schema import HumanMessage, SystemMessage, AIMessage

print("--- Interacting with Models ---")

if openai_available:
    # --- 使用 Chat Model (推荐) ---
    print("\nUsing ChatOpenAI (gpt-3.5-turbo by default):")
    try:
        chat = ChatOpenAI(temperature=0.7) # temperature 控制创造性 (0=确定性, >1=更随机)
        
        # 简单的人类消息
        human_message = HumanMessage(content="Explain the concept of 'gradient descent' in one sentence.")
        ai_response = chat([human_message])
        print(f"Response to simple query: {ai_response.content}")
        print(f"Response type: {type(ai_response)}")
        
        # 包含系统消息和多轮对话
        messages = [
            SystemMessage(content="You are a helpful assistant that translates English to French."),
            HumanMessage(content="Translate: 'Hello, how are you?'")
        ]
        ai_response_french = chat(messages)
        print(f"\nResponse to translation query: {ai_response_french.content}")
        
        # 可以将 AI 的回答加入消息列表继续对话
        messages.append(ai_response_french)
        messages.append(HumanMessage(content="And 'Thank you'?"))
        ai_response_thanks = chat(messages)
        print(f"Response to 'Thank you': {ai_response_thanks.content}")

    except Exception as e:
        print(f"Error interacting with ChatOpenAI: {e}")
        print("This might be due to invalid API key, network issues, or OpenAI service status.")
        
    # --- (可选) 使用旧的 LLM 接口 --- 
    # print("\nUsing OpenAI LLM (text-davinci-003 - might be deprecated):")
    # try:
    #     llm = OpenAI(temperature=0.9)
    #     text_prompt = "What is the capital of France?"
    #     completion = llm(text_prompt)
    #     print(f"LLM completion for '{text_prompt}': {completion.strip()}")
    # except Exception as e:
    #     print(f"Error interacting with OpenAI LLM: {e}")
        
else:
    print("Skipping Model interaction examples because OpenAI API Key is not configured.")

## 3. Prompts: 使用 Prompt Templates

Prompt Templates 帮助我们动态地、一致地构建给 LLM 的指令 (Prompt)。

*   **`PromptTemplate`**: 用于简单的、包含几个变量的模板。
*   **`ChatPromptTemplate`**: 用于构建聊天消息列表的模板，可以包含系统消息、人类消息模板等。

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate

print("--- Using Prompt Templates ---")

# --- PromptTemplate (for LLMs) --- 
template_llm = "Tell me a short joke about a {subject}."
prompt_llm = PromptTemplate(input_variables=["subject"], template=template_llm)
formatted_prompt_llm = prompt_llm.format(subject="computer")
print(f"Formatted LLM prompt:\n{formatted_prompt_llm}")

# --- ChatPromptTemplate (for Chat Models) --- 
system_template = "You are a helpful assistant who writes concise summaries in {language}."
human_template = "Summarize the following text: {text}"

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 格式化聊天提示
formatted_chat_messages = chat_prompt.format_messages(
    language="Spanish", 
    text="LangChain provides modules for models, prompts, indexes, memory, chains, and agents. It aims to simplify building complex applications powered by large language models."
)

print("\nFormatted Chat Messages:")
for msg in formatted_chat_messages:
    print(f"  Type: {type(msg).__name__}, Content: '{msg.content[:50]}...'" if len(msg.content) > 50 else f"  Type: {type(msg).__name__}, Content: '{msg.content}'")
    
# 可以在与 Chat Model 交互时直接使用 format_prompt().to_messages()
# formatted_prompt_value = chat_prompt.format_prompt(language="Spanish", text="...")
# messages_for_llm = formatted_prompt_value.to_messages()

## 4. LCEL (LangChain Expression Language): 构建简单的 Chain

LCEL 是 LangChain 的核心，它提供了一种声明式的方式来组合不同的 LangChain 组件 (如 Prompt, Model, Output Parser)，使用类似 Python 管道 (`|`) 的语法。

**好处**: 
*   **可组合性**: 轻松连接组件。
*   **流式处理 (Streaming)**: 支持流式输出。
*   **批处理 (Batching)**: 高效处理多个输入。
*   **异步支持 (Async)**。
*   **可观测性 (Observability)**: 与 LangSmith 等工具集成良好。

In [ ]:
from langchain.schema.output_parser import StrOutputParser

print("--- Using LCEL to build a simple chain ---")

# 定义一个简单的链: Prompt -> Chat Model -> String Output Parser
# 1. Prompt Template (使用之前的 chat_prompt)
# 2. Chat Model (使用之前的 chat 实例)
# 3. Output Parser (将 AI Message 转换为字符串)
output_parser = StrOutputParser()

if openai_available:
    # 定义链
    # The | operator chains the components
    summarization_chain = chat_prompt | chat | output_parser
    print(f"Chain created: {summarization_chain}")
    
    # 调用链 (使用 invoke)
    input_dict = {
        "language": "English",
        "text": "The weather today is sunny with a high of 25 degrees Celsius. There is a slight breeze from the west."
    }
    try:
        summary_result = summarization_chain.invoke(input_dict)
        print("\nInvoking the summarization chain:")
        print(f"Input: {input_dict['text']}")
        print(f"Output Summary (English): {summary_result}")
        
        # 尝试不同的语言
        input_dict_fr = {**input_dict, "language": "French"}
        summary_result_fr = summarization_chain.invoke(input_dict_fr)
        print(f"\nOutput Summary (French): {summary_result_fr}")
        
    except Exception as e:
        print(f"\nError invoking chain: {e}")
        
else:
    print("Skipping LCEL chain example (OpenAI Key not available).")

## 5. Indexes: 文档加载、分割、嵌入和向量存储 (基础)

为了让 LLM 能够利用外部或私有数据（实现 RAG），我们需要对这些数据进行处理：

1.  **加载 (Load)**: 使用 `DocumentLoader` 从各种来源（文件、网页、数据库等）加载数据为 `Document` 对象。
2.  **分割 (Split)**: 使用 `TextSplitter` 将长文档分割成更小的、语义相关的块 (chunks)。
3.  **存储 (Store)**: 
    *   使用 **Text Embedding Model** 将每个块转换为数值向量 (embedding)。
    *   将文本块和对应的 embedding 存储到**向量存储 (Vector Store)** 中，以便高效检索。

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings # 需要 OpenAI key
# from langchain.embeddings import HuggingFaceEmbeddings # 使用 Hugging Face 模型 (本地或 Hub)
from langchain.vectorstores import FAISS # 使用 FAISS 作为向量存储
# from langchain.vectorstores import Chroma # 另一个流行的本地向量存储

print("--- Indexing Example (Load, Split, Embed, Store) ---")

# --- 0. 准备示例文本文件 ---
sample_text_content = """
LangChain is a framework for developing applications powered by language models.
It enables applications that are: Data-aware and Agentic.
Data-aware: connect a language model to other sources of data.
Agentic: allow a language model to interact with its environment.
The main value props of LangChain are: Components and Use-Cases.
Components: modular abstractions for working with language models.
Use-Cases: end-to-end chains for common applications like RAG and Agents.
LCEL, or LangChain Expression Language, is a declarative way to compose chains.
"""
sample_text_path = "langchain_sample.txt"
with open(sample_text_path, "w") as f:
    f.write(sample_text_content)
print(f"Sample text file '{sample_text_path}' created.")

# --- 1. 加载文档 ---
loader = TextLoader(sample_text_path)
documents = loader.load()
print(f"\nLoaded {len(documents)} document(s).")
print(f"First document content (preview):\n{documents[0].page_content[:100]}...")

# --- 2. 分割文档 ---
# RecursiveCharacterTextSplitter 尝试按段落、句子、单词等分割，保持语义完整性
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150, # 每个块的最大字符数
    chunk_overlap=20  # 块之间的重叠字符数
)
split_docs = text_splitter.split_documents(documents)
print(f"\nDocument split into {len(split_docs)} chunks.")
print("First chunk example:")
print(split_docs[0].page_content)

# --- 3. 创建 Embedding 模型 --- 
embeddings = None
if openai_available:
    try:
        embeddings = OpenAIEmbeddings()
        print("\nOpenAIEmbeddings model created.")
        # 示例：获取第一个块的嵌入向量
        # vector = embeddings.embed_query(split_docs[0].page_content)
        # print(f"Embedding vector dimension: {len(vector)}")
    except Exception as e:
        print(f"Error creating OpenAI embeddings: {e}")
else:
    print("\nSkipping embedding creation (OpenAI key unavailable).")
    # # 或者使用 Hugging Face 的模型 (需要安装 sentence-transformers)
    # try:
    #     from langchain.embeddings import HuggingFaceEmbeddings
    #     model_name = "sentence-transformers/all-MiniLM-L6-v2"
    #     embeddings = HuggingFaceEmbeddings(model_name=model_name)
    #     print(f"HuggingFaceEmbeddings model '{model_name}' created.")
    # except ImportError:
    #     print("HuggingFaceEmbeddings requires 'sentence-transformers'. Install it.")
    # except Exception as e:
    #     print(f"Error creating HuggingFace embeddings: {e}")

# --- 4. 创建并填充向量存储 (Vector Store) --- 
vector_store = None
if embeddings and split_docs:
    try:
        # 使用 FAISS (内存向量存储)
        print("Creating FAISS vector store from documents...")
        vector_store = FAISS.from_documents(split_docs, embeddings)
        print(f"FAISS vector store created. Index contains {vector_store.index.ntotal} vectors.")
        
        # 可以保存和加载 FAISS 索引
        # vector_store.save_local("faiss_index_langchain")
        # new_vector_store = FAISS.load_local("faiss_index_langchain", embeddings)
        
    except Exception as e:
        print(f"Error creating or using FAISS vector store: {e}")
else:
    print("\nSkipping vector store creation (embeddings or documents unavailable).")

# 清理文本文件
if os.path.exists(sample_text_path):
    os.remove(sample_text_path)

## 6. Retrievers: 从索引中检索信息

向量存储通常提供一个 `.as_retriever()` 方法，返回一个 Retriever 对象。Retriever 负责根据用户查询，从向量存储中找出最相关的文档块。

In [ ]:
print("--- Using Retriever --- ")

if vector_store:
    retriever = vector_store.as_retriever(search_kwargs={"k": 2}) # 获取最相关的 2 个块
    print("Retriever created from FAISS vector store.")
    
    query = "What is LCEL?"
    try:
        relevant_docs = retriever.get_relevant_documents(query)
        print(f"\nQuery: '{query}'")
        print(f"Retrieved {len(relevant_docs)} relevant documents:")
        for i, doc in enumerate(relevant_docs):
            print(f"  Doc {i+1}: {doc.page_content.replace('\n', ' ')}")
            # print(f"    Metadata: {doc.metadata}") # 通常包含来源信息
    except Exception as e:
         print(f"Error during retrieval: {e}")
else:
    print("Vector store not available, skipping retriever example.")
    retriever = None # Define retriever as None if not created

## 7. 构建简单的 RAG (Retrieval-Augmented Generation) Chain

RAG 结合了信息检索和语言模型生成。基本流程：
1.  接收用户问题。
2.  使用 Retriever 查找相关的文档块。
3.  将问题和检索到的上下文信息一起输入到 Prompt Template。
4.  将格式化后的 Prompt 发送给 LLM。
5.  LLM 基于上下文生成答案。

LCEL 使这个流程的构建非常直观。

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

print("--- Building a Simple RAG Chain --- ")

if openai_available and retriever:
    # 定义 RAG 的 Prompt Template
    rag_template = """Answer the question based only on the following context:
    {context}
    
    Question: {question}
    
    Answer:"""
    rag_prompt = ChatPromptTemplate.from_template(rag_template)
    
    # 定义 RAG 链
    # 1. 输入: 一个包含 "question" 键的字典
    # 2. RunnablePassthrough 将 "question" 传递下去
    # 3. 同时，使用 retriever 获取相关 context (输入也是 "question")
    # 4. 将 question 和 context 组合成 prompt 的输入
    # 5. 调用 prompt template
    # 6. 调用 LLM (chat model)
    # 7. 解析输出
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()} 
        | rag_prompt 
        | chat # Use the ChatOpenAI instance from earlier
        | StrOutputParser()
    )
    
    print("RAG chain created.")

    # 调用 RAG 链
    user_question = "What is LangChain Expression Language?"
    print(f"\nInvoking RAG chain with question: '{user_question}'")
    try:
        rag_answer = rag_chain.invoke(user_question)
        print(f"\nRAG Answer:\n{rag_answer}")
        
        user_question_2 = "What are the main value props of LangChain?"
        print(f"\nInvoking RAG chain with question: '{user_question_2}'")
        rag_answer_2 = rag_chain.invoke(user_question_2)
        print(f"\nRAG Answer:\n{rag_answer_2}")
        
    except Exception as e:
        print(f"Error invoking RAG chain: {e}")
else:
    print("Skipping RAG chain example (OpenAI key or Retriever unavailable).")

## 8. (简介) Memory

Memory 组件用于在 Chain 或 Agent 的多次调用之间存储和检索信息，最常见的用途是维持对话历史。

*   **`ConversationBufferMemory`**: 存储完整的对话历史。
*   **`ConversationBufferWindowMemory`**: 只存储最近的 K 轮对话。
*   **`ConversationSummaryMemory`**: 使用 LLM 动态总结对话历史。

通常将 Memory 对象添加到 Chain (如 `ConversationChain`) 或 Agent 中。

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

print("\n--- Memory Introduction ---")

if openai_available:
    # 创建带 Memory 的对话链
    memory = ConversationBufferMemory()
    conversation = ConversationChain(
        llm=chat, # Use the ChatOpenAI instance from earlier
        memory=memory,
        verbose=False # Set to True to see the full prompt with history
    )
    print("ConversationChain with memory created.")
    
    # 进行对话
    print("\nSimulating conversation with memory:")
    try:
        response1 = conversation.predict(input="Hi there! My name is Bob.")
        print(f"AI Response 1: {response1}")
        
        response2 = conversation.predict(input="What is my name?")
        print(f"AI Response 2: {response2}") # AI 应该能记住名字
        
        # 查看 Memory 中的内容
        print("\nCurrent Memory Buffer:")
        print(memory.buffer)
        
    except Exception as e:
        print(f"Error during conversation chain: {e}")
else:
    print("Skipping Memory example (OpenAI key not available).")

## 9. (简介) Agents & Tools

Agent 使用 LLM 来**决定**采取哪些行动以及行动的顺序。这些行动可以由 **Tools** 提供，例如：
*   Google 搜索
*   Python REPL
*   计算器
*   数据库查询
*   调用其他 Chain 或 API

**基本流程**: 
1.  定义 Agent 可以使用的 Tools。
2.  初始化 Agent Executor，传入 LLM、Tools 和 Prompt（通常使用特定的 Agent Prompt）。
3.  运行 Agent Executor，提供用户输入。
4.  Agent 会进行思考、选择工具、执行工具、观察结果，并循环此过程，直到任务完成或达到限制。

构建 Agent 涉及更多细节，如 Prompt 工程、选择合适的 Agent 类型 (如 ReAct, OpenAI Functions Agent) 等。

In [ ]:
# Agent 示例通常需要额外的库 (如 google-search-results) 或 API Key
# 这里仅作概念说明
print("\n--- Agents & Tools Introduction ---")
print("Agents use an LLM to decide which 'Tools' (like search, calculator, other chains) to use.")
print("Example (conceptual):")
print("1. Define tools (e.g., a search tool, a calculator tool).")
print("2. Initialize an Agent Executor with the LLM and tools.")
print("3. Run the executor with a complex query (e.g., 'What was the weather in London yesterday and what is that temperature in Fahrenheit?').")
print("4. The agent would decide to use the search tool first, then the calculator tool.")

# from langchain.agents import load_tools, initialize_agent, AgentType
# if openai_available:
#     try:
#         # Requires 'pip install google-search-results'
#         # Requires SERPAPI_API_KEY environment variable
#         tools = load_tools(["serpapi", "llm-math"], llm=chat) # Example tools
#         agent = initialize_agent(tools, chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
#         # agent.run("Who is the current CEO of OpenAI? What is their age raised to the power of 0.5?")
#         print("\n(Agent execution code commented out due to external dependencies/keys)")
#     except Exception as e:
#         print(f"\nCould not initialize or run agent (missing dependencies/keys?): {e}")

## 总结

LangChain 是一个功能非常丰富的框架，极大地简化了构建基于 LLM 的复杂应用程序的过程。

**关键要点：**
*   核心在于模块化组件 (Models, Prompts, Indexes, Memory, Chains, Agents, Tools)。
*   **LCEL** 是组合这些组件的现代、声明式方式。
*   **RAG** (通过 Indexes 和 Retrievers 实现) 是让 LLM 利用外部知识的关键模式。
*   **Memory** 用于构建有状态的对话应用。
*   **Agents** 赋予 LLM 执行动作和与环境交互的能力。

LangChain 的生态系统仍在快速发展，掌握其核心组件和 LCEL 是构建强大 LLM 应用的基础。官方文档 ([https://python.langchain.com/](https://python.langchain.com/)) 提供了大量深入的指南和示例。

In [ ]:
# Final cleanup of the sample text file if it still exists
if os.path.exists(sample_text_path):
    os.remove(sample_text_path)
    print(f"Cleaned up {sample_text_path}")
# Cleanup potential FAISS index files (if saving was uncommented)
# import shutil
# if os.path.exists("faiss_index_langchain"): 
#     shutil.rmtree("faiss_index_langchain")